In [9]:
%matplotlib inline
from neurotools.nlab import *
import os
import struct

In [3]:
os.listdir('.')

['q4-v1.png',
 'q5-v2.xcf',
 'q5-v4.png',
 'Music.desktop',
 '20170807_notes_meeting_gerrit',
 'Dropbox.desktop',
 'DoubleClassLinux.tar.gz',
 'DoubleClass.zip',
 'q5.png',
 'q5-v1.xcf',
 'q4-v3.xcf',
 'note to stuart',
 'Workspace.desktop',
 'q5-v4.xcf',
 'q6-v1.xcf',
 'Python.desktop',
 'Terminal.desktop',
 'q4.png',
 'q4-v1_hueloop',
 '20170809_hacking_the_GIF_pallet.ipynb',
 'q4-v1-animated.gif',
 'index.html~',
 'Moonbase.desktop',
 '090756971.pdf',
 'Hobbies',
 'DoubleClass',
 'q6-v3-loop-small.xcf',
 'q6.png',
 'q6-v3.xcf',
 'q6-v2_hueloop',
 'DoubleClassLinux',
 '20170807_notes_meeting_gerrit~',
 'herb allergies?~',
 'q6-v3-loop-small.gif',
 'q6-v3-loop.xcf',
 'Github',
 'q6-v2.png',
 'q5-v4_hueloop',
 'q6-v2.xcf',
 'q4-v2.xcf',
 'note to stuart~',
 'q5-v4.gif',
 'Dump.desktop',
 'q5-v3.xcf',
 'index.png',
 'MRule_CV_August_2017.pdf',
 '.ipynb_checkpoints']

In [4]:
fn = 'q5-v4.gif'
Nhue      = 12
huestart  = 2

See http://www.matthewflickinger.com/lab/whatsinagif/bits_and_bytes.asp

Objective is to permute GIF color pallet to perform a hue rotation / animation on a specially prepared input. We need only read as far as the color table.

The input pallet used hsa 14 colors. The first two are shades of grey. The following 12 are a hue loop that we'd like to permute.

We need to output a new file that is binary identitcal to the original, with the exception of the rotated subset of the color pallet.

In [52]:
# Blind copy sanity check
outfile = "sanity.gif"
with open(fn,'rb') as f:
    with open(outfile,'wb') as fout:
        fout.write(f.read())

In [59]:
with open(fn, "rb") as f:

    # Get header and extract and print flags, fields, etc
    header = f.read(13)
    form = header[0:][:6].decode('ascii')
    w = struct.unpack("H",header[6:][:2])[0]
    h = struct.unpack("H",header[8:][:2])[0]
    print('%d x %d %s'%(w,h,form))
    flags = struct.unpack("B",header[10:][:1])[0]
    bgci  = struct.unpack("B",header[11:][:1])[0]
    pasp  = struct.unpack("B",header[12:][:1])[0]
    gcts  = flags&0b00000111
    sort  = flags&0b00001000>>3
    cres  = flags&0b01110000>>4
    gctf  = flags&0b10000000>>7
    print('Global color table flag',gctf)
    print('Color resolution',cres)
    print('Sort flag',sort)
    print('Size of global color table',gcts)
    print('Background color index',bgci) # No. Colors?
    print('Pixel aspect ratio',pasp)

    # Get color pallet information
    Ncolors = 2**(gcts+1)
    print('No. colors used  ',bgci)
    print('No. colors stored',Ncolors)
    colordata = []
    for i in range(Ncolors):
        RGB = f.read(3)
        print('Color %d'%i,RGB)
        colordata+=[[RGB]]
    
    # get remainder of file
    imagedata = f.read()

colordata = array(colordata)
hues      = array(colordata[huestart:huestart+Nhue])

# now open an output file and do the rotation(s)
basename = '.'.join(fn.split('.')[:-1])
for i in range(Nhue):
    fout  = basename+'_rotate_%d.gif'%i
    colordata[huestart:huestart+Nhue] = np.roll(hues,i,axis=0)    
    with open(fout,"wb") as f:
        f.write(header)
        f.write(colordata)
        f.write(imagedata)
    print('Saved',fout)

2084 x 2084 GIF89a
Global color table flag 1
Color resolution 3
Sort flag 1
Size of global color table 3
Background color index 14
Pixel aspect ratio 0
No. colors used   14
No. colors stored 16
Color 0 b'\xfe\xff\xfc'
Color 1 b'\xa3\xa3\xa3'
Color 2 b'\xb8\x1f\xff'
Color 3 b'kz\xff'
Color 4 b'\x00\xaa\xff'
Color 5 b'\x00\xd0\xd1'
Color 6 b'\x00\xd6m'
Color 7 b'\x93\xd8\x00'
Color 8 b'\xc9\xd8\x03'
Color 9 b'\xff\xa9\x0c'
Color 10 b'\xff\x8d\x00'
Color 11 b'\xffbB'
Color 12 b'\xf2\x11\xc5'
Color 13 b'\xdf\x00\xf2'
Color 14 b'\xff\x00\xff'
Color 15 b'\xff\x00\xff'
Saved q5-v4_rotate_0.gif
Saved q5-v4_rotate_1.gif
Saved q5-v4_rotate_2.gif
Saved q5-v4_rotate_3.gif
Saved q5-v4_rotate_4.gif
Saved q5-v4_rotate_5.gif
Saved q5-v4_rotate_6.gif
Saved q5-v4_rotate_7.gif
Saved q5-v4_rotate_8.gif
Saved q5-v4_rotate_9.gif
Saved q5-v4_rotate_10.gif
Saved q5-v4_rotate_11.gif
